Arquivo de análise usando a frequência: analise_doc_term_matriz_freq_with_header_discrete_AND.txt
Arquivo de análise usando tf-idf: analise_doc_term_matriz_tfidf_with_header_discrete_AND.txt

#### Sequência de leitura do arquivo
1- Nome do arquivo que gerei para a análise<br />
2- Nome do arquivo que o professor gerou para a análise<br />
3- Nome do arquivo base usado para gerar os grupos<br />
4- Número total Colunas (atributos)<br />
5- Número total de Linhas (exemplos/amostras)<br />
6- Nome da métrica de avaliação<br />
7- Operador entre características usado pelo SSDP<br />
##### // A partir daqui se repete até a última classe //
8- Quebra de linha<br />
9- Número identificador da classe avaliada<br />
10- Quebra de linha<br />
11- O conjunto de regras gerado a partir do PageRank<br />
12- Quebra de linha<br />
13- O conjunto de regras gerado a partir do total<br />

In [1]:
import pandas as pd
import numpy as np

In [14]:
headerList = []

dfCommunity = pd.DataFrame()
dfWordsOnRules = pd.DataFrame()

def colectDataFromFile(fileName, isOnlyToCountFeaturesInRules):
    readHeader = False
    dfPageRank = pd.DataFrame(columns=['community', 'rules'])    
    dfTotalSet = pd.DataFrame(columns=['community', 'rules'])
    
    dfPageRankFeaturesCounting = pd.DataFrame(columns=['pr_features_counting'])
    dfTotalSetFeaturesCounting = pd.DataFrame(columns=['ts_features_counting'])

    with open('../testingPageRank/' + fileName) as f:
        if 'freq' in fileName:
            filenameIdentifier = 'freq'
        else:
            filenameIdentifier = 'tfidf'
            
        lastID = ''
        countSets = 0
        
        dfWordsOnRulesPageRank = pd.DataFrame(columns=['caracteristicasPageRank'])
        dfWordsOnRulesTotalSet = pd.DataFrame(columns=['caracteristicasTotalSet'])

        for line in f:
            cleanLine = line.replace('\n','').strip()

            if not readHeader:
                headerList.append(cleanLine)
            else:
                if cleanLine.isdigit():
                    lastID = cleanLine

                elif '{' in cleanLine:
                    cleanLine = cleanLine[1:-1]
                    
                    listDasCaracteristicas = [[strWord] for strWord in cleanLine.split(',')]

                    if countSets < 20:
                        dfPageRank.loc[len(dfPageRank)] = cleanLine
                        dfPageRankFeaturesCounting.loc[len(dfPageRankFeaturesCounting)] = len([x.strip() for x in cleanLine.split(',')])
                        
                        if dfWordsOnRulesPageRank.empty:
                            dfWordsOnRulesPageRank = pd.DataFrame(listDasCaracteristicas,
                                                                  columns=['caracteristicasPageRank'])
                            
                        else:
                            dfTemp = pd.DataFrame(listDasCaracteristicas, columns=['caracteristicasPageRank'])
                            dfWordsOnRulesPageRank = pd.concat([dfWordsOnRulesPageRank, dfTemp], ignore_index=True)
                        
                    else:
                        dfTotalSet.loc[len(dfTotalSet)] = cleanLine
                        dfTotalSetFeaturesCounting.loc[len(dfTotalSetFeaturesCounting)] = len([x.strip() for x in cleanLine.split(',')])
                        
                        if dfWordsOnRulesTotalSet.empty:
                            dfWordsOnRulesTotalSet = pd.DataFrame(listDasCaracteristicas,
                                                                  columns=['caracteristicasTotalSet'])
                            
                        else:
                            dfTemp = pd.DataFrame(listDasCaracteristicas, columns=['caracteristicasTotalSet'])
                            dfWordsOnRulesTotalSet = pd.concat([dfWordsOnRulesTotalSet, dfTemp], ignore_index=True)

                    countSets += 1

                if countSets > 0 and len(dfTotalSet) == 20:
                    # interseção
                    if not isOnlyToCountFeaturesInRules:
                        dfIntersection = pd.merge(dfPageRank, dfTotalSet, how='inner', on=['rules'])
                        dfIntersection.drop(['community_x', 'community_y'], axis=1, inplace=True)
                        dfIntersection.columns = ['rulesIntersection']
                    
                    # as diferenças entre as regras
                    setA = set(dfPageRank['rules'])
                    setB = set(dfTotalSet['rules'])
                    
                    listAB = list(setA - setB)
                    listBA = list(setB - setA)

                    listNanA = [np.nan] * (len(dfPageRank['rules']) - len(listAB))
                    listNanB = [np.nan] * (len(dfTotalSet['rules']) - len(listBA))
                    
                    dfPageRank.columns = ['community', 'PageRank']
                    dfTotalSet.columns = ['community', 'TotalSet']
                    dfPageRank['community'] = lastID
                    dfTotalSet['community'] = lastID
                    
                    if not isOnlyToCountFeaturesInRules:
                        dfCommunity = pd.concat([dfPageRank, dfTotalSet.drop(['community'], axis=1), dfIntersection], axis=1)
                        
                        dfCommunity['onlyOnPageRank'] = listAB + listNanA
                        dfCommunity['onlyOnTotalSet'] = listBA + listNanB
                    else:
                        dfCommunity = pd.concat([dfPageRank, dfPageRankFeaturesCounting, dfTotalSetFeaturesCounting,
                                                 dfTotalSet.drop(['community'], axis=1)], axis=1)
                    
                    # reinicializando os data frames
                    dfPageRank = pd.DataFrame(columns=['community', 'rules'])
                    dfTotalSet = pd.DataFrame(columns=['community', 'rules'])
                    
                    dfPageRankFeaturesCounting = pd.DataFrame(columns=['pr_features_counting'])
                    dfTotalSetFeaturesCounting = pd.DataFrame(columns=['ts_features_counting'])
    
                    countSets = 0

                    if not isOnlyToCountFeaturesInRules:
                        dfCommunity.to_csv('filesResults/rules_community_' + lastID + '_' + filenameIdentifier + '.csv',
                                           header=True, index=False, sep=';')
                    else:
                        dfCommunity.to_csv('filesResults/rules_community_features_counting_' + lastID + '_' + filenameIdentifier + '.csv',
                                       header=True, index=False, sep=';')
            
                    # Agora tratando as palavras presentes nas regras
                    dfWordsOnRulesPageRank['freqWordPageRank'] = dfWordsOnRulesPageRank.\
                        groupby('caracteristicasPageRank')['caracteristicasPageRank'].transform('count')
                    dfWordsOnRulesPageRank.sort_values('freqWordPageRank', ascending=False, inplace=True)
                    dfWordsOnRulesPageRank.drop_duplicates(subset='caracteristicasPageRank', keep='first', inplace=True)
                    dfWordsOnRulesPageRank.reset_index(inplace=True)
                    dfWordsOnRulesPageRank.drop('index',inplace=True,axis=1)
                    dfWordsOnRulesPageRank.freqWordPageRank = dfWordsOnRulesPageRank.freqWordPageRank.\
                        apply(int)
                    
                    dfWordsOnRulesTotalSet['freqWordTotalSet'] = dfWordsOnRulesTotalSet.\
                        groupby('caracteristicasTotalSet')['caracteristicasTotalSet'].transform('count')
                    dfWordsOnRulesTotalSet.sort_values('freqWordTotalSet', ascending=False, inplace=True)
                    dfWordsOnRulesTotalSet.drop_duplicates(subset='caracteristicasTotalSet', keep='first', inplace=True)
                    dfWordsOnRulesTotalSet.reset_index(inplace=True)
                    dfWordsOnRulesTotalSet.drop('index',inplace=True,axis=1)
                    dfWordsOnRulesTotalSet.freqWordTotalSet = dfWordsOnRulesTotalSet.freqWordTotalSet.\
                        apply(int)
                    
                    dfWordsOnRules = pd.concat([dfWordsOnRulesPageRank, dfWordsOnRulesTotalSet], axis=1)
            
#                     dfWordsOnRules.to_csv('filesResults/words_rules_community_' + lastID + '_' + filenameIdentifier + '.csv',
#                                           header=True, index=False, sep=';')
            
                    dfCommunity = pd.DataFrame()
                
                    dfWordsOnRules = pd.DataFrame()
                    
            if cleanLine == 'AND' or cleanLine == 'OR':
                readHeader = True


##### Testing data collect

In [15]:
colectDataFromFile('analise_doc_term_matriz_freq_with_header_discrete_AND.txt', True)
colectDataFromFile('analise_doc_term_matriz_tfidf_with_header_discrete_AND.txt', True)